# Scrap them all !

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request, response


In [5]:
%qtconsole

In [171]:
def clean_ministres_fonctions(fonction_ministres):
    fonction_ministres_clean =  [x for x in fonction_ministres if any([len(x) < 5, '[' not in x ])]
    fonction_ministres_clean = '#'.join(fonction_ministres_clean).split('##')
    fonction_ministres_clean = [x.replace('#', ' / ') for x in fonction_ministres_clean]

    return fonction_ministres_clean

def clean_ministres_wiki(wiki_ministre) : 
    wiki_ministres_clean = [x for x in wiki_ministre if ('France' not in x) & 
                            ('jpg' not in x) & ('Fichier' not in x) & any(['(' in x or 'Conway' in x, len(x) <= 35]) &  # Philipe martin (homme politique, ...)
                            ('gauche' not in x) & ('Outre-' not in x) & ('Ville' not in (x)) &
                            ('Logement' not in x) & ('Budget' not in x)  & ('Walwari' not in x) &
                            ('#cite' not in x) & ('tiquette' not in x) & ('cologistes' not in x) & ('cologie' not in x)]
    print('%s ministres trouvés' % (len(wiki_ministres_clean)))
    
    return wiki_ministres_clean
# le any(['(' in x or 'Conway' in x, len(x) <= 36]) sert à conserver certains wiki atypiques (Conway = très lon, Philippe Martin = (homme polituque..))

def get_my_gouv(link_gouv):
    # Request
    header = {'User-Agent': 'Mozilla/5.0'} # Needed to prevent 403 error on Wikipedia
    page = request.Request(link_gouv, headers=header)
    html = request.urlopen(page) 
    soup = BeautifulSoup(html, "lxml")
    cells = soup.findAll("table", {"class": "wikitable sortable"})
    #  Init
    wiki_ministres = []
    fonction_ministres = [] 
    n_ministres = []
    n_min = 0
    # Scraping
    for cell in cells: # Tableau principal
        cells_href = cell.findAll("a", href=True)
        for c_href in cells_href: # éléments de chaque tableau
            wiki_ministres.append(c_href['href'])
            fonction_ministres.append(c_href.get_text())
            n_ministres.append(n_min)
            n_min += 1
    wiki_ministres_clean = clean_ministres_wiki(wiki_ministres)
    fonction_ministres_clean = clean_ministres_fonctions(fonction_ministres)
    return (fonction_ministres_clean, wiki_ministres_clean)

In [180]:
dict_holland = {'holland_ayrault_1': "https://fr.wikipedia.org/wiki/Gouvernement_Jean-Marc_Ayrault_(1)",
                'holland_ayrault_2': "https://fr.wikipedia.org/wiki/Gouvernement_Jean-Marc_Ayrault_(2)",
                'holland_valls_1': "https://fr.wikipedia.org/wiki/Gouvernement_Manuel_Valls_(1)",
                'holland_valls_2': "https://fr.wikipedia.org/wiki/Gouvernement_Manuel_Valls_(2)" }

for keys, values in dict_holland.items():
    gouv = pd.DataFrame()
    (fonction_ministres_clean, wiki_ministres_clean) = get_my_gouv(values)
    gouv[keys + "_WIKI"] = wiki_ministres_clean
    gouv[keys + "_FONCTIONS"] = fonction_ministres_clean
    gouv.to_csv('M:\datasciences\data\ministres\%s.csv' % (keys), sep=';')

35 ministres trouvés
32 ministres trouvés
39 ministres trouvés
38 ministres trouvés


In [173]:
(fonction_ministres_clean, wiki_ministres_clean) = get_my_gouv("https://fr.wikipedia.org/wiki/Gouvernement_Manuel_Valls_(2)")

39 ministres trouvés


In [174]:
len(fonction_ministres_clean)

39

In [175]:
len(wiki_ministres_clean)

39

## Test
Vérifions qu'il ya bien autant de ministres que de fonction. Visuellement.

In [176]:
pd.options.display.max_colwidth = 300
df = pd.DataFrame()
df['wiki_ministres'] = wiki_ministres_clean
df1 = pd.DataFrame()
df1['fonction_ministres'] = pd.Series(fonction_ministres_clean)

df.merge(df1, left_index=True, right_index=True, how='outer')  # changer 'left' ou 'right' pour identifier un éventuel pb
#df['fonctions_ministres'] = fonction_ministres_clean

,wiki_ministres,fonction_ministres
0,/wiki/Manuel_Valls,/ Premier ministre / Manuel Valls / PS
1,/wiki/Jean-Marc_Ayrault,Ministre des Affaires étrangères et du Développement international / Jean-Marc Ayrault / PS
2,/wiki/S%C3%A9gol%C3%A8ne_Royal,"Ministre de l'Environnement, de l'Énergie et de la Mer, chargée des relations internationales sur le climat / Ségolène Royal / PS"
3,/wiki/Najat_Vallaud-Belkacem,Ministre de l'Éducation nationale / de l'Enseignement supérieur et de la Recherche / Najat Vallaud-Belkacem / PS
4,/wiki/Michel_Sapin,Ministre des Finances et des Comptes publics / Michel Sapin / PS
5,/wiki/Marisol_Touraine,Ministre des Affaires sociales et de la Santé / Marisol Touraine / PS
6,/wiki/Jean-Yves_Le_Drian,Ministre de la Défense / Jean-Yves Le Drian / PS
7,/wiki/Jean-Jacques_Urvoas,"Garde des Sceaux, ministre de la Justice / Jean-Jacques Urvoas / PS"
8,/wiki/Myriam_El_Khomri,"Ministre du Travail, de l'Emploi, de la Formation professionnelle et du Dialogue social / Myriam El Khomri / PS"
9,/wiki/Jean-Michel_Baylet,"Ministre de l'Aménagement du territoire, de la Ruralité et des Collectivités territoriales / Jean-Michel Baylet / PRG"


## Looping the ministres

In [19]:
for ministre in link_ministres_clean[:2]:
    page_ministre = "https://fr.wikipedia.org" + ministre
    header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
    page = request.Request(page_ministre, headers=header)
    html = request.urlopen(page)
    soup = BeautifulSoup(html, "lxml")

In [22]:
print(soup.find('table', {'class': 'infobox_v2'}))

<table class="infobox_v2">
<tr>
<td class="entete defaut" colspan="2" style="background-color:#6688FF;color:#FFFFFF">Laurent Fabius</td>
</tr>
<tr>
<td colspan="2" style="text-align:center; line-height: 1.5em;"><a class="image" href="/wiki/Fichier:Laurent_Fabius_January_2015.jpg"><img alt="Laurent Fabius, en janvier 2015." data-file-height="1182" data-file-width="888" height="293" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/1b/Laurent_Fabius_January_2015.jpg/220px-Laurent_Fabius_January_2015.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/1b/Laurent_Fabius_January_2015.jpg/330px-Laurent_Fabius_January_2015.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/1b/Laurent_Fabius_January_2015.jpg/440px-Laurent_Fabius_January_2015.jpg 2x" width="220"/></a><br/>
Laurent Fabius, en janvier 2015.</td>
</tr>
<tr>
<th colspan="2" style="padding:4px;text-align:center;background-color:#6688FF;color:#FFFFFF">Fonctions</th>
</tr>
<tr>
<th colspan="2" style="padding:4px;